In [1]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

In [2]:
# CHANGE ME!
TASK_ID = 'changing_mean'
AUTHOR = 'g'


In [13]:
task = Task(
    id=TASK_ID,
    name="Changing mean",
    description="Return index of the first element you can remove that does not change the mean of the list by more than 0.1",
    plan=[
        {
        "id": "remove",
        "description": "remove",
    }, {
        "id": "orig",
        "description": "mean of the list",
    }, {
        "id": "new",
        "description": "change the mean",
    }, 
        {
        "id": "diff",
        "description": "by more than 0.1",
    }, 
        {
        "id": "first",
        "description": "index of the first element",
    }, 
    ],
    sample_input={
       "vals":[{"id":0,"value":1.0},{"id":1,"value":2.0},{"id":2,"value":3.0},{"id":3,"value":1.0}],
    },
    sample_output=1,
    category="Basic",
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [4]:
imp = replace(prototype,
    language='python-imperative',
    source='''
def get_mean(ls):
  sum = 0
  for e in ls:
    sum += e['value']
  return sum/len(ls)

def changing_mean(vals):
  start_mean = get_mean(vals)
  for i,elem in enumerate(vals):
    new_ls = vals[:i] + vals[i+1:]
    mean = get_mean(new_ls)
    if abs(mean - start_mean) < 0.1:
      return i
  return None
''').load_plan()
imp.execute(task)
imp.save()

In [11]:
func = replace(prototype,
    language='python-functional',
    source='''
def changing_mean(vals):
  start_mean = sum([l['value'] for l in vals])/len(vals)
  diff = lambda m, sm: abs(m - sm)
    
  for i,elem in enumerate(vals):
    new_ls = [x['value'] for x in vals if x != elem]
    mean = sum(new_ls)/len(new_ls)
    if diff(mean,start_mean) < 0.1:
      return i
  return None
''').load_plan()
func.execute(task)
func.save()

In [6]:
pandas = replace(prototype,
    language='python-pandas',
    source='''
def changing_mean(vals):
  mean = vals.value.mean()
  mean_without = vals.apply(lambda row: vals[vals.id != row.id].value.mean(), axis=1)
  return vals[(mean_without - mean).abs() < 0.1].id.tolist()
''').load_plan()
pandas.execute(task)
pandas.save()

In [10]:
#NOTE: Issue with float conversion
datalog = replace(prototype,
    language='datalog',
    source='''.decl avg_except(I:number, Avg:float)    
avg_except(I, Avg) :- 
  vals(I, _),
  N = sum 1.0 : { vals(J, _), I != J },
  Total = sum V : { vals(J, V), I != J },
  Avg = Total / N.
  
changing_mean(I) :-
  vals(I, _),
  N = sum 1.0 : vals(_, _),
  Total = sum V : vals(_, V),
  Avg = Total / N,
  avg_except(I, AvgExcept),
  ((Avg > AvgExcept, Avg - AvgExcept < 0.1);
   (Avg < AvgExcept, AvgExcept - Avg < 0.1)).''').load_plan()
datalog.execute(task)
datalog.save()

Segmentation violation signal in rule:
avg_except(I,(sum  V1 : { vals(J, V1),I != J }/sum 1.0 : { vals(J,_),I != J })) :- 
   vals(I,_).
in file /tmp/tmpd10n7wr4/program.dl [6:1-10:19]



CalledProcessError: Command 'souffle -F. -D. program.dl' returned non-zero exit status 1.

In [8]:
#NOTE issue with getting average of dataset without current value
sql = replace(prototype,
    language='sql',
    source='''SELECT id
FROM vals v1
WHERE ABS((
  SELECT AVG(value)
  FROM vals v2
  WHERE v1.id != v2.id
) - (SELECT AVG(value) FROM vals)) < 0.1''').load_plan()
sql.execute(task)x
sql.save()

In [23]:
program = datalog
program.widget(task)

Output()

CodeWidget(program='{"task": "changing_mean", "language": "datalog", "plan": {"orig": [{"line": 9, "start": 2,…